## Code samples

The above is frontmatter that might be needed by devportal when exported; ignore it for now. The rest of this notebook would be visible to the customer, as is. First, let's see Python work.

In [7]:
print("hello world")

hello world


For me it does; it says hello world below the python cell. Yay! One could change hello world to any other string to see dynamic running code, in its most fundamental form.
## Splunk Observability Cloud

The code below sends metric data to your Splunk Obvervability Cloud instance. Within 'https://ingest.us1.signalfx.com', replace 
_us1_ with your realm, _j195EsqEpXQgHQhec9P2ZA_ with your access token, and _you_ with your alias. If you don't do this...it actually sends metric data to _my_ instance so thanks for that.

In [8]:
import time
import signalfx

st = time.time()

sum_x = 0

for i in range(1000000):
    sum_x += i

# wait for 3 seconds
time.sleep(3)

print('Sum of first 1 million numbers is:', sum_x)

et = time.time()

elapsed = et - st

with signalfx.SignalFx(ingest_endpoint='https://ingest.us1.signalfx.com').ingest('j195EsqEpXQgHQhec9P2ZA') as sfx:
    sfx.send(
        gauges=[
          {
                "metric": "execution.time",
                "dimensions": { "host": "you" },
                "value": et - st
          }
        ])

print("Time sent to Splunk O11y")

Sum of first 1 million numbers is: 499999500000
Time sent to Splunk O11y


## Splunk Python SDK

The following code processes data from a Splunk Enterprise search using the Splunk Python SDK.

If you are running this notebook locally and receive something like "ModuleNotFoundError: No module named 'splunklib'" that means you must first install the missing package. Meaning, from your command prompt enter ``pip install splunk-sdk``. If you don't have pip, you're likely not a Python coder yet; you need to first run ``python get-pip.py``. If you don't have Python yet, you're _definitely_ not a Python coder so go install it and become one (in that case, you're not quite yet the audience for python samples in a python-based notebook.) And as the notebook will tell you: you may need to restart the Notebook kernel to use updated packages (__Kernel__, __Restart Kernel__).

In [9]:
import splunklib.client as client
import splunklib.results as results
from time import sleep

Now its time to connect to your already running Splunk Enterprise instance! In the following code, replace pking and password with your own Splunk Enterprise login credentials

In [10]:
HOST = "localhost"
PORT = 8089
USER_NAME = "pking"
PASSWORD = "password"
service = client.connect(host=HOST, port=PORT, username=USER_NAME, password=PASSWORD)

Now, let's do something real using SPL to get data from Splunk Enterprise.

In [11]:
job = service.jobs.create('search index=_internal ( log_level=ERROR OR log_level=WARN ) | stats count')
while not job.is_done():
    sleep(.2)
rr = results.JSONResultsReader(job.results(output_mode='json'))
for result in rr:
    print(result)
assert rr.is_preview == False

{'count': '392204'}


There you go: you have the count of existing warnings in your internal Splunk index! For me, when I run it, the count was 360921. Next time it should differ. But...data! That's the point.

To make it interactive, see the [Jupyter Binder form](https://mybinder.org/v2/gh/PKing70/DocsFromJupyter/HEAD?labpath=DFJ.ipynb) of this Notebook hosted by mybinder.org or click its neat badge: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/PKing70/DocsFromJupyter/HEAD?labpath=DFJ.ipynb) 